# SCC-DFTB repulsive potential generation (2-body)

Input: DFT_DB.db DFTB_DB.db 

Output: CCS parameters to be used in conjunction with the SK-tables generated in gen_SK.ipynb and CCS_DB.db 



In [1]:
PARAUTOMATIK_PATH='/Users/tjams20/Documents/repos/ParAutomatik/'   # Add your installation path here

In [2]:
import os, sys, warnings
sys.path.append(PARAUTOMATIK_PATH+'/pgm/CCS/')
sys.path.append(PARAUTOMATIK_PATH+'/pgm/python_scripts')
import json
import pandas as pd
import itertools as it
from collections import OrderedDict, defaultdict
import numpy as np
from ase import Atoms
from ase import io
from ase.calculators.neighborlist import NeighborList
import ase.db as db
from tqdm import tqdm 

from ccs.common.exceptions import ScriptError 
from ccs.ase_calculator.ccs_ase_calculator import CCS
from ccs import ccs_fetch, ccs_fit

import matplotlib.pyplot as plt
import seaborn as sns


base_dir=os.getcwd()
os.chdir(base_dir)
print('Base dir is:', base_dir)
warnings.filterwarnings('ignore')


/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.9/site-packages/ase/calculators/neighborlist.py:5: UserWarning: Moved to ase.neighborlist
  warnings.warn('Moved to ase.neighborlist')


Base dir is: /Users/tjams20/Documents/repos/ParAutomatik/pgm/Jupyter_notebooks


In [3]:
#generate data generates trainingset in the format for CCS (structures.json)
cwd = os.getcwd()
if not os.path.isdir(cwd+"/CCS/"):
    os.mkdir(cwd+"/CCS/")
os.chdir(cwd+"/CCS/")

ccs_fetch(mode='CCS', R_c=6.0, Ns='all', DFT_DB='../DFT_DB.db', DFTB_DB='../DFTB_DB.db')

os.chdir(base_dir) 

100%|████████████████████████████████████████| 999/999 [00:04<00:00, 216.95it/s]


In [4]:
#DEFINE INPUT
os.chdir(cwd+"/CCS/")

input={
    "General": {
        "interface": "CCS"
    },
    "Twobody": {
                "Xx-Xx": {
                        "Rcut": 6.0,
                        "Resolution": 0.05,
                        "Swtype": "rep"
                }
        }
}
#SAVE TO FILE
with open('input.json', 'w') as f:
    json.dump(input, f, indent=8)
    
#RUN FIT
ccs_fit("input.json")
os.chdir(base_dir)
print("done")

Generating one-body information from training-set.
    Added elements:  ['C', 'H', 'Li', 'O']
Generating two-body potentials from one-body information.
C C
    Adding pair: C-C
C H
    Adding pair: C-H
C Li
    Adding pair: C-Li
C O
    Adding pair: C-O
H C
H H
    Adding pair: H-H
H Li
    Adding pair: H-Li
H O
    Adding pair: H-O
Li C
Li H
Li Li
    Adding pair: Li-Li
Li O
    Adding pair: Li-O
O C
O H
O Li
O O
    Adding pair: O-O


Finding optimum switch: 100%|█████████████████████| 1/1 [00:01<00:00,  1.47s/it]


DEBUG: Current pair C-C has 98 knots and [[ 0.          0.16666667  1.         ... 94.         95.
  47.83333333]
 [ 0.          0.          0.16666667 ... 93.         94.
  47.33333333]
 [ 0.          0.          0.         ... 92.         93.
  46.83333333]
 ...
 [ 0.          0.          0.         ...  0.16666667  1.
   0.83333333]
 [ 0.          0.          0.         ...  0.          0.16666667
   0.33333333]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]] coefficients.
DEBUG: Current pair C-H has 103 knots and [[  0.           0.16666667   1.         ...  99.         100.
   50.33333333]
 [  0.           0.           0.16666667 ...  98.          99.
   49.83333333]
 [  0.           0.           0.         ...  97.          98.
   49.33333333]
 ...
 [  0.           0.           0.         ...   0.16666667   1.
    0.83333333]
 [  0.           0.           0.         ...   0.           0.16666667
    0.33333333]
 [  0.           0.           0.         ..

IndexError: index 1 is out of bounds for axis 0 with size 1

# Analysis

In [ ]:
os.chdir(base_dir)

struct=db.connect("/Users/tjams20/Documents/repos/ParAutomatik/Electrolyte/DFT_DB.db")
db_ccs=db.connect("CCS_DB.db")

print(len(struct))

os.chdir(base_dir+"/CCS/")
with open('CCS_params.json', 'r') as f:
    CCS_params = json.load(f)

for row in tqdm(struct.select(), total=len(struct)):  
    structure=row.toatoms()
    structure.calc=CCS(CCS_params=CCS_params)
    structure.get_potential_energy()
    structure.get_forces()
    db_ccs.write(structure,key=row.key,CCS=True)

os.chdir(base_dir)

In [ ]:
#Read Data
import matplotlib.pyplot as plt
import seaborn as sns
dbname_DFT='/Users/tjams20/Documents/repos/ParAutomatik/Electrolyte/DFT_DB.db'
# dbname_DFTB='DFTB_DB.db'
dbname_CCS='CCS_DB.db'
# db_dftb=db.connect(dbname_DFTB)
db_ccs=db.connect(dbname_CCS)
db_dft=db.connect(dbname_DFT)
epred=[];edft=[];eccs=[];
for row in tqdm(db_dft.select(), total=len(db_dft)):
    structure=row.toatoms()
    natoms=len(structure.get_atomic_numbers())
    id=str(row.key)
    # edftb.append(db_dftb.get('key='+id).energy/natoms)
    eccs.append(db_ccs.get('key='+id).energy/natoms)
    edft.append(db_dft.get('key='+id).energy/natoms) 
    # edftb_css.append(db_dftb.get('key='+id).energy/natoms + db_ccs.get('key='+id).energy/natoms)


In [ ]:
#plot Energies

df_plt = pd.DataFrame({"A":edft, "B":eccs})
print(df_plt)

plt.figure(figsize = (10,10))
sns.scatterplot(df_plt, x='A', y='B', s=50, alpha=0.8, color='r')

plt.plot([np.min(edft),np.max(edft)], [np.min(edft),np.max(edft)], 'k-', lw=2.5)
plt.legend(labels=["DFTB+CCS"], fontsize=30 )
plt.xlabel('E$_{DFT}$/atom (eV)',fontsize=30)
plt.ylabel('E$_{DFTB+CCS}$',fontsize=30)
plt.xlim(np.min(edft),np.max(edft))
plt.ylim(np.min(edft),np.max(edft))
plt.savefig('DFTvsDFTB+CCS.png')
plt.show()